<a href="https://colab.research.google.com/github/fenz/notebook_workspace/blob/master/TPU_simple_bench.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%%capture
!pip install watermark

In [0]:
%load_ext watermark
%watermark -p tensorflow,numpy -m

tensorflow 1.12.0
numpy 1.14.6

compiler   : GCC 8.2.0
system     : Linux
release    : 4.14.65+
machine    : x86_64
processor  : x86_64
CPU cores  : 2
interpreter: 64bit


In [0]:
import os
import time
import tensorflow as tf

def calc(n):
  img = tf.random_normal((128, 100, 100, 3))
  def body(_):
    result = tf.layers.conv2d(img, 32, 7)
    result = tf.reduce_sum(result)
    return result

  return tf.contrib.tpu.repeat(n[0], body, [0.0])


session = tf.Session('grpc://' + os.environ['COLAB_TPU_ADDR'])
try:
  print('Initializing TPU...')
  session.run(tf.contrib.tpu.initialize_system())

  for i in [1, 10, 100, 500]:
    tpu_ops = tf.contrib.tpu.batch_parallel(calc, [[i] * 8], num_shards=8)
    print('Warming up...')
    session.run(tf.global_variables_initializer())
    session.run(tpu_ops)
    print('Profiling')
    start = time.time()
    session.run(tpu_ops)
    end = time.time()
    elapsed = end - start
    print(i, elapsed)
finally:
  session.run(tf.contrib.tpu.shutdown_system())
  session.close()

Initializing TPU...
Warming up...
Profiling
1 0.009667634963989258
Warming up...
Profiling
10 0.03323030471801758
Warming up...
Profiling
100 0.2860987186431885
Warming up...
Profiling
500 1.4136693477630615
